In [1]:
import json
import mysql.connector

from mysql.connector import Error

In [4]:
def create_connection(config):
    try:
        connection = mysql.connector.connect(**config)
        if connection.is_connected():
            print("Соединение с базой данных успешно установлено.")
        return connection
    except Error as e:
        print(f"Ошибка подключения: {e}")
        return None
    
with open("params.json") as f:
    dict_conn = json.load(f)

connection = create_connection(dict_conn)

Соединение с базой данных успешно установлено.


### Запрос 1
Вывести информацию обо всех деревьях общепринятого названия вида "Дуб" в парке "Центральный парк". В запросе вывести общепринятое название, научное название вида, дату посадки.

In [5]:
def query_trees_by_species_and_park(connection, species_name, park_name):
    """Запрос деревьев указанного вида в определенном парке"""
    try:
        cursor = connection.cursor(prepared=False)
        query = """
            SELECT ts.common_name, ts.scientific_name, t.planting_date
            FROM trees t
            JOIN tree_species ts ON t.species_id = ts.species_id
            JOIN parks p ON t.park_id = p.park_id
            WHERE ts.common_name = ? AND p.name = ?;
        """
        cursor.execute(query, (species_name, park_name))
        result = cursor.fetchall()
        print(f"Деревья вида '{species_name}' в '{park_name}':")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [6]:
query_trees_by_species_and_park(connection, 'Oak', 'Central Park')

Ошибка выполнения запроса: Not all parameters were used in the SQL statement


### Запрос 2
Вывести информацию о волонтерах, ухаживающих за деревьями, включая количество выполненных работ по уходу за деревьями. Если волонтер не выполнял работ - вывести 0. В запросе вывести имя волонтера, фамилию, телефон, количество выполненных работ (столбец назвать count).

In [6]:
def query_volunteers_maintenance_count(connection):
    """Запрос количества работ, выполненных волонтерами"""
    try:
        cursor = connection.cursor()
        query = """
            SELECT v.first_name, v.last_name, v.phone, COUNT(m.maintenance_id) AS count
            FROM volunteers v
            LEFT JOIN maintenance m ON v.volunteer_id = m.volunteer_id
            GROUP BY v.volunteer_id;
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Волонтеры и количество их работ:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [7]:
query_volunteers_maintenance_count(connection)

Волонтеры и количество их работ:
('John', 'Doe', '555-1234', 3)
('Jane', 'Smith', '555-5678', 1)
('John', 'Doe', '+123456789', 3)
('John', 'Doe', '+123456789', 0)
('John', 'Doe', '+123456789', 0)


### Запрос 3
Для каждого парка вывести информацию о количестве деревьев каждого вида, посаженных в парке. Отсутствующие в парке виды деревьев не выводить. Упорядочить по названию парка, виду дерева. В запросе вывести название парка, общепринятое название вида, количество деревьев (столбец назвать count).

In [8]:

def query_tree_count_by_park_and_species(connection):
    """Запрос количества деревьев по паркам и видам"""
    try:
        cursor = connection.cursor()
        query = """
            SELECT p.name AS park_name, ts.common_name, COUNT(t.tree_id) AS count
            FROM trees t
            JOIN parks p ON t.park_id = p.park_id
            JOIN tree_species ts ON t.species_id = ts.species_id
            GROUP BY p.name, ts.common_name
            ORDER BY p.name, ts.common_name;
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Количество деревьев по паркам и видам:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [9]:
query_tree_count_by_park_and_species(connection)

Количество деревьев по паркам и видам:
('Central Park', 'Oak', 5)


### Запрос 4
Вывести информацию обо всех волонтерах, выполнивших больше 5 работ, упорядочить по фамилии, имени. В запросе вывести столбцы: имя волонтера, фамилия, количество выполненных работ (столбец назвать count).

In [10]:
def query_active_volunteers(connection, min_count):
    """Запрос волонтеров с количеством работ больше указанного значения"""
    try:
        cursor = connection.cursor(prepared=True)
        query = """
            SELECT v.first_name, v.last_name, COUNT(m.maintenance_id) AS count
            FROM volunteers v
            JOIN maintenance m ON v.volunteer_id = m.volunteer_id
            GROUP BY v.volunteer_id
            HAVING COUNT(m.maintenance_id) > ?
            ORDER BY v.last_name, v.first_name;
        """
        cursor.execute(query, (min_count,))
        result = cursor.fetchall()
        print(f"Волонтеры с количеством работ больше {min_count}:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [11]:
query_active_volunteers(connection, 1)

Волонтеры с количеством работ больше 1:
('John', 'Doe', 3)
('John', 'Doe', 3)


### Запрос 5
Вывести список парков, в которых посажено максимальное число деревьев. В запросе вывести идентификатор парка, название парка.

In [12]:

def query_park_with_max_trees(connection):
    """Запрос парка с максимальным количеством деревьев"""
    try:
        cursor = connection.cursor()
        query = """
            WITH TreeCount AS (
                SELECT park_id, COUNT(tree_id) AS tree_count
                FROM trees
                GROUP BY park_id
            )
            SELECT p.park_id, p.name
            FROM parks p
            JOIN TreeCount tc ON p.park_id = tc.park_id
            WHERE tc.tree_count = (SELECT MAX(tree_count) FROM TreeCount);
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Парк с максимальным количеством деревьев:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [13]:
query_park_with_max_trees(connection)

Парк с максимальным количеством деревьев:
(1, 'Central Park')


### Запрос 6
Посчитать количество проведенных в 2024 году работ по уходу за деревьями для каждого парка (учесть парки, в которых работ не проводилось). Вывести информацию о статистике проведенных работ. В запросе вывести минимальное (столбец назвать min), среднее (avg) и максимальное (max) количество проведенных работ (результат запроса - одна строка).

In [14]:
def query_work_statistics_2024(connection):
    """Статистика работ по паркам за 2024 год"""
    try:
        cursor = connection.cursor()
        query = """
            WITH park_work_counts AS (
                SELECT
                    p.park_id,
                    COUNT(m.maintenance_id) AS work_count
                FROM parks p
                LEFT JOIN trees t ON p.park_id = t.park_id
                LEFT JOIN maintenance m ON t.tree_id = m.tree_id AND YEAR(m.maintenance_date) = 2023
                GROUP BY p.park_id
            )
            SELECT
                MIN(work_count) AS min,
                AVG(work_count) AS avg,
                MAX(work_count) AS max
            FROM park_work_counts;
        """
        cursor.execute(query)
        result = cursor.fetchall()
        print("Статистика работ по паркам за 2024 год:")
        for row in result:
            print(row)
    except Error as e:
        print(f"Ошибка выполнения запроса: {e}")

In [15]:
query_work_statistics_2024(connection)

Статистика работ по паркам за 2024 год:
(0, Decimal('0.2857'), 1)


In [16]:
if connection.is_connected():
    connection.close()
    print("Соединение с базой данных закрыто.")

Соединение с базой данных закрыто.
